# DSBA-HCIP 6160: Data Acquisition and Ingestion for Data Analytics

### Author: Naomi Thammadi

## Data Acquisition - Retrieving details of Deceased Patients using FHIR REST API

### Import necessary Libraries

In [1]:
#import necessary libraries
!pip install requests
!pip install pymysql
import requests
import os
import json
import csv
import pandas as pd
import numpy as np
import math

#libraries to make a connection to mysql
import sqlalchemy
from sqlalchemy import create_engine
import pymysql

### Functions 

In [2]:
#function to get the element for phone number from the json 
def get_correct_phone_data(arr):
    #data can be NaN or a list
    #below conditions check for that to get the right data within the list
    if type(arr) == list:
        if(len(arr)>1 and 'value' not in arr[0].keys()):
            return arr[1]
        else:
            return arr[0]
    elif math.isnan(arr) == False:
        return arr[0]
    else:
        return 'unknown'

In [3]:
#function to clean patient data to convert lists to string and get right data from the json
def clean_patient_data(df):
    df['Name_JSON']=df['Name_JSON'].str[0]
    df['Address_JSON']=df['Address_JSON'].str[0]
    df['Phone_JSON']=df['Phone_JSON'].apply(lambda x:get_correct_phone_data(x))

In [4]:
#function to get name, address and phone details from the json
def concat_patient_data(df):
    #get name details and drop unnecessary columns
    name_df=df['Name_JSON'].apply(json.dumps).apply(json.loads).apply(pd.Series)
    name_df['Given Name']=[' '.join(map(str, l)) for l in name_df['given']]
    name_df.drop(['use','fhir_comments','text','given','prefix'], axis = 1, inplace = True, errors='ignore')
    
    #get address details, format them and drop unnecessary columns
    address_df=df['Address_JSON'].apply(json.dumps).apply(json.loads).apply(pd.Series)
    address_df.drop([0,'district','text','type','use','period','extension','fhir_comments'], axis = 1, inplace = True, errors='ignore')
    address_df.fillna('', inplace=True)
    #combine street address into one line
    address_df['line'] = [','.join(map(str, l)) for l in address_df['line']]
    #combine street address, city, state, country into one column separated by (,)
    address_df['Address']=address_df.apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    address_df.Address = address_df.Address.replace({',,,,,': 'unknown'})
    address_df.rename(columns={'city': 'City', 'state': 'State', 'country': 'Country'}, inplace=True)
    
    #get phone details and drop unnecessary columns
    phone_df=df['Phone_JSON'].apply(json.dumps).apply(json.loads).apply(pd.Series)
    phone_df['Phone']=phone_df['value']
    phone_df.drop([0,'rank','system','use','value'], axis = 1, inplace = True, errors='ignore')
    phone_df.fillna('-', inplace=True)
    
    #concatenate all these dataframes into one and return it
    df=pd.concat([df,name_df,address_df,phone_df],axis=1)
    
    return df

In [5]:
#function to arrange the data correctly to display
def arrange_patient_data(df):
    df['Family Name']=df['family']
    df.drop(['Name_JSON','Address_JSON','Phone_JSON','family','line','postalCode','period','id'], axis = 1, inplace = True, errors='ignore')
    df.fillna('-', inplace=True)
    df['Date of Death'] = df['Date of Death'].replace({'-': 'unknown'})
    df=df[['Patient Id','Given Name','Family Name','Gender','Date of Birth','Date of Death','Address','City','State','Country','Phone']]
    return df

In [6]:
#function to search for patients based on search string
def search_patients():
    #url to make the API callout
    url='http://hapi.fhir.org/baseR4/Patient?deceased=true'
    response = requests.get(url)
    #format the response to json
    resp=response.json() 
    #store into a dataframe
    data=pd.json_normalize(resp['entry'])
    
    patient_df=data.filter(['resource.id','resource.name','resource.gender','resource.birthDate','resource.deceasedDateTime','resource.address','resource.telecom','resource.deceasedBoolean'], axis=1)
    patient_df.columns=['Patient Id','Name_JSON','Gender','Date of Birth','Date of Death', 'Address_JSON','Phone_JSON','Deceased']
    
    #call functions to run the necessary data preprocessing steps
    clean_patient_data(patient_df)
    patient_df=concat_patient_data(patient_df)
    patient_df=arrange_patient_data(patient_df)
    
    return patient_df

In [7]:
#function to display search results in a html
def on_search_clicked(b):
    search_patients.data=''
    results=search_patients()
    
    #set the dataframe into a HTML table
    search_patients.data= results.to_html();
    
    display(HTML(search_patients.data))

## Display Patient Details

In [8]:
from ipywidgets import widgets
from IPython.core.display import display, HTML

display(HTML('<h4>Click on "Display Patients" to show all the deceased patients.</h4>'))
search_btn=widgets.Button(description="Display Patients",button_style="success")
display(search_btn)
search_btn.on_click(on_search_clicked)



Button(button_style='success', description='Display Patients', style=ButtonStyle())

,Patient Id,Given Name,Family Name,Gender,Date of Birth,Date of Death,Address,City,State,Country,Phone
0,gtp101,Herbert,Hoover,male,1990-07-04,2019-07-30T02:34:06.000-05:00,"123 Main Street North,Everytown,USA,99999,",Everytown,USA,,8885551234
1,1059,Maria MN123,Max Mountbaton,male,2001-10-06,2015-06-17T12:38:39.000+05:30,"L0111 L0112 L0113,L1112,Seattle,WA,98052,US",Seattle,WA,US,1311111111
2,1139,testname MN123,LN123,male,2001-10-05,2015-06-17T12:38:39.000+05:30,"L0111 L0112 L0113,L1112,Seattle,WA,98052,US",Seattle,WA,US,1311111110
3,1190,FN123 MN123,LN123,male,2001-10-05,2015-06-17T12:38:39.000+05:30,"L0111 L0112 L0113,L1112,Seattle,WA,98052,US",Seattle,WA,US,1311111111
4,24739,Tina A,Roy,male,2001-10-10,2010-11-11T11:12:14.000+05:30,"Street comp1 comp2,AddLine2,Seattle,WA,98052,USA",Seattle,WA,USA,123-456-1111
5,30309,Steven,Johnston,male,1940-11-01,unknown,"9938 American Place,FORT LAUDERDALE,FORT LAUDERDALE,FL,33319,US",FORT LAUDERDALE,FL,US,931-583-7931
6,31678,FN3062 MN3062,test_fn,male,2001-10-10,2010-11-11T11:12:14.000+05:30,"Street comp1 comp2,AddLine2,Seattle,WA,98052,USA",Seattle,WA,USA,123-456-1111
7,40052,Peter James,Chalmers,male,1974-12-25,2019-02-16T23:42:00,"534 Erewhon St,PleasantVille,Vic,3999,",PleasantVille,Vic,,-
8,40562,Peter James,Chalmers,male,1974-12-25,2015-02-14T13:42:00+10:00,"534 Erewhon St,PleasantVille,Vic,3999,",PleasantVille,Vic,,+1-305-555-6401x1234
9,63070,Katerina,Nguyen,female,1979-08-20,unknown,"Unit 7,76 Clydesdale St,Como,WA,6152,Australia",Como,WA,Australia,0403823832


## Data Ingestion - Transfer Patient details to MySQL database

In [9]:
#read the csv files and convert into dataframe
parent_dir='./Data/'
file_name='patient_data.csv'
patients_df=pd.read_csv(parent_dir+file_name)
patients_df.head()

,patient_id,given_name,family_name,gender,date_of_birth,date_of_death,city,state,phone,death_by_natural_cause,alcohol_intake,nicotine_intake,race,bmi,health_issues
0,7949473,Gaynor,Titta,Female,6/4/46,10/22/16 15:42,West Palm Beach,Florida,561-553-2724,0,Never,Never,Asian,Normal,1
1,9662622,Oona,Pusill,Female,11/26/65,5/17/16 22:33,Hollywood,Florida,954-178-3132,0,Often,Often,Alaska Native,Over weight,1
2,6716847,Yetta,Taysbil,Male,9/14/47,11/16/07 10:39,Jersey City,New Jersey,201-765-7538,0,Never,Never,African American,Under weight,1
3,8349189,Farrah,Bartles,Male,12/8/97,2/18/00 1:18,Murfreesboro,Tennessee,615-130-9597,0,Often,Often,Alaska Native,Over weight,0
4,3790901,Richardo,Hedau,Bigender,7/11/77,12/24/93 11:05,Reston,Virginia,571-153-4428,1,Never,Occasionally,Native Hawaiian,Normal,1


In [10]:
patients_df.shape

(10000, 15)

In [11]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   patient_id              10000 non-null  int64 
 1   given_name              10000 non-null  object
 2   family_name             10000 non-null  object
 3   gender                  10000 non-null  object
 4   date_of_birth           10000 non-null  object
 5   date_of_death           10000 non-null  object
 6   city                    10000 non-null  object
 7   state                   10000 non-null  object
 8   phone                   10000 non-null  object
 9   death_by_natural_cause  10000 non-null  int64 
 10  alcohol_intake          10000 non-null  object
 11  nicotine_intake         10000 non-null  object
 12  race                    10000 non-null  object
 13  bmi                     10000 non-null  object
 14  health_issues           10000 non-null  int64 
dtypes: 

In [12]:
#convert date fields in the correct format to insert into the database
patients_df['date_of_birth']=pd.to_datetime(patients_df.date_of_birth)
patients_df['date_of_death']=pd.to_datetime(patients_df.date_of_death)

## Connect to MySQL

In [13]:
#function to connect to MySQL Database
def connect_to_db(hostname,dbname,uname,pwd):
    # Credentials to database connection
    # hostname: usually localhost or any other location where the data is hosted
    # dbname: database schema name
    # uname: username for authentication
    # pwd: password for authentication

    # Create SQLAlchemy engine to connect to MySQL Database
    sqlEngine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))
    #dictionary of the sql connection engine and connection message
    connection_results={'engine': sqlEngine, 'message': ''}
    
    #connect to the database
    try:
        dbConnection = sqlEngine.connect()
        connection_results.update({'message':'Connection Successful'})
    except Exception as ex:
        connection_results.update({'message':'Connection Failed. Error: '+str(ex)})
    finally:
        dbConnection.close()
        
    return connection_results;

In [14]:
#function to call connect_to_db on button click
def on_connect_clicked(b):
    connect_to_db.data=''
    connect_to_db.data=connect_to_db("localhost","Patient_DB","dbuser","db1234").get('message')
    display(HTML(connect_to_db.data))

In [15]:
#connect to the database
display(HTML('<h4>Click on "Connect to MySQL" to test the connection to the database</h4>'))
connect_btn=widgets.Button(description="Connect to MySQL",button_style="success")
display(connect_btn)
connect_btn.on_click(on_connect_clicked)


Button(button_style='success', description='Connect to MySQL', style=ButtonStyle())

## Transfer data to MySQL

In [16]:
#function to add rows to Patients table 
def transfer_to_db():
    #get the patient details
    message=''
    #check for size of the dataframe before transferring data to MySQL
    if len(patients_df) > 0:
        #call the method to connect to MySQL
        connection=connect_to_db("localhost","Patient_DB","dbuser","db1234")
        sqlEngine=connection.get('engine')
        #table to insert into
        tbname='patients'
        
        #insert records into the table
        try:
            dbConnection = sqlEngine.connect()
            #df.to_sql() will insert the dataframe values into MySQL table
            #since table is already created, if_exists will append the results to it
            #index=False so that dataframe index is not added to MySQL table
            patients_df.to_sql(tbname, sqlEngine, index=False, if_exists='append') 
            message='Data Successfully transfered to Patients table'
        except Exception as ex:
            message='Transfer Failed. Error: '+str(ex)
        finally:
            dbConnection.close()
            
    else:
        message='No data to transfer to Patients table'
        
    return message

In [17]:
#function to call transfer_to_db on button click
def on_transfer_clicked(b):
    transfer_to_db.data=''
    transfer_to_db.data=transfer_to_db()
    display(HTML(transfer_to_db.data))

In [18]:
#transfer data to the database
display(HTML('<h4>After Data Preprocessing, click on "Transfer" to write data to Patients table</h4>'))
transfer_btn=widgets.Button(description="Transfer",button_style="success")
display(transfer_btn)
transfer_btn.on_click(on_transfer_clicked)


Button(button_style='success', description='Transfer', style=ButtonStyle())

## Retrieve data from MySQL

In [19]:
#function to retrieve selected rows from Patients table 
def retrieve_from_db():
    #call the method to connect to MySQL
    connection=connect_to_db("localhost","Patient_DB","dbuser","db1234")
    sqlEngine=connection.get('engine')
    ResultSet = pd.DataFrame()
    
    try:
        dbConnection = sqlEngine.connect()
        metadata = sqlalchemy.MetaData()
        #get patients table
        patients = sqlalchemy.Table('patients', metadata, autoload=True, autoload_with=sqlEngine)
        query = sqlalchemy.select([patients]).where(sqlalchemy.and_(patients.columns.gender=="Female",patients.columns.state=="Florida")).order_by(sqlalchemy.asc(patients.columns.family_name)) 
                #Equivalent to 'SELECT * FROM patients WHERE gender=Female AND state=Florida order by family_name'
        ResultProxy = dbConnection.execute(query).fetchall()
        ResultSet=pd.DataFrame(ResultProxy)
        ResultSet.columns = patients.columns.keys()
        
    except Exception as ex:
        print('Retrieval Failed. Error: '+str(ex))
    
    finally:
        dbConnection.close()
        
    return ResultSet;

In [20]:
#function to call retreive_from_db on button click
def on_retrieve_clicked(b):
    retrieve_from_db.data=''
    retrieve_from_db.data=retrieve_from_db().to_html()
    display(HTML(retrieve_from_db.data))

In [21]:
#retrieve patients from database
display(HTML('<h4>Retrieve Patients table</h4>'))
retrieve_btn=widgets.Button(description="Get Patients",button_style="success")
display(retrieve_btn)
retrieve_btn.on_click(on_retrieve_clicked)

Button(button_style='success', description='Get Patients', style=ButtonStyle())

### Assignment:

<ol>
    <li>Create a Patients table in MySQL</li>
    <li>Load <b>patients_data.csv</b> into python</li>
    <li>Transfer patients data to MySQL</li>
    <li>Retrieve patients data from MySQL to Tableau</li>
    <li>Run some analytics on MySQL/Tableau</li>
</ol>